In [1]:
import getpass
import os
from pydantic import BaseModel, Field, field_validator
from typing import Optional
from langgraph.graph import Graph
from threading import Timer
from typing import Optional, Annotated
import time
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("llmgsk_6QCRZZGxVnJEd9PVrxmfWGdyb3FYLMhr7XESbMf6LJeElvRKiy0C")

In [2]:
llm = ChatGroq(groq_api_key=os.environ['GROQ_API_KEY'], model_name="llama3-70b-8192")

In [3]:
class ChatGroqWrapper:
    def __init__(self, groq_api_key, model_name):
        self.groq_api_key = groq_api_key
        self.model_name = model_name

    def send_message(self, prompt):
        print(f"LLM ({self.model_name}): {prompt}")

In [4]:
class UserData(BaseModel):
    user_id: str
    start_time: float = Field(default_factory=lambda: time.time())
    prompted: bool = False
    completed: bool = False
    rated: bool = False
    rating_requested: bool = False
    reminder_sent: int = 0

In [5]:
class GraphState(BaseModel):
    user_data: UserData
    incoming_msg: Optional[str] = None
    awaiting_response: bool = False
    proceed_to_encuesta: bool = False
    information_provided: bool = False
    user_response: bool = False

In [6]:
@field_validator('incoming_msg')
def validate_message(cls, v):
        if v and "@" in v and any(char.isdigit() for char in v):
            return v
        return v

In [7]:
user_states = {}

In [8]:
def validado(state: GraphState):
    user_id = state.user_data.user_id
    llm.send_message(f"{user_id}, por favor proporciona tu documento y correo electrónico.")
    state.user_data.prompted = True
    state.awaiting_response = True
    return "timer_5min"

In [9]:
def timer_5min(state: GraphState):
    user_id = state.user_data.user_id
    
    def reminder():
        if not state.user_data.completed:
            state.user_data.reminder_sent += 1
            llm.send_message(f"Recordatorio {state.user_data.reminder_sent}: No olvides proporcionar tu documento y correo electrónico.")
            
            if state.user_data.reminder_sent < 2:
                Timer(120, reminder).start()

    Timer(120, reminder).start()
    
    def check_completion():
        if state.user_data.completed:
            state.information_provided = True
            graph.transition_to("informacion")  # Transición a 'informacion'
        else:
            state.proceed_to_encuesta = True
            graph.transition_to("encuesta")  # Transición a 'encuesta'
    
    Timer(300, check_completion).start()


In [10]:
def informacion(state: GraphState):
    user_id = state.user_data.user_id
    llm.send_message(f"{user_id}, gracias por proporcionar tu información. Ahora te proporcionaremos la información solicitada.")
    return "encuesta"

In [11]:
def encuesta(state: GraphState):
    user_id = state.user_data.user_id
    llm.send_message(f"{user_id}, ¿cómo calificarías la información recibida? Responde con una cantidad de estrellas (1-5).")
    state.rating_requested = True

In [12]:
graph = Graph()

In [13]:
graph.add_node("validado", validado)
graph.add_node("timer_5min", timer_5min)
graph.add_node("informacion", informacion)
graph.add_node("encuesta", encuesta)

In [14]:
graph.add_edge("validado", "timer_5min")
graph.add_edge("informacion", "encuesta")

In [15]:
def run_graph(user_id, incoming_msg=None):
    user_data = UserData(user_id=user_id)
    initial_state = GraphState(user_data=user_data, incoming_msg=incoming_msg)

    if user_id not in user_states:
        user_states[user_id] = user_data
        graph.run(start_node="validado", state=initial_state)

    if incoming_msg and "@" in incoming_msg and any(char.isdigit() for char in incoming_msg):
        initial_state.user_data.completed = True
        initial_state.awaiting_response = False
        print(f"{user_id} ha proporcionado la información correcta.")

In [16]:
app = graph.compile()

ValueError: Node `validado` is not reachable